pca_fasttext_200features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Fasttext/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Fasttext/fasttext_withstopwordchangelevels_200_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,0,9.468417,10.481529,-0.480864,-2.673449,1.761255,-3.715870,5.291796,4.194867,4.319843,...,-1.712980e-07,-7.989175e-07,-3.233028e-07,4.095239e-07,1.498896e-07,-3.064581e-07,-8.746509e-08,-2.664218e-07,2.489527e-07,-6.001070e-10
1,1,-0.383868,2.043244,1.357626,0.252145,4.158366,-1.648929,6.765302,2.074958,0.964495,...,-1.163429e-07,3.039201e-07,4.276332e-08,-5.187756e-09,2.005606e-07,7.780325e-08,-1.824773e-07,-1.162510e-07,-7.168037e-08,-2.535856e-07
2,2,-2.731018,2.971175,-0.750384,-0.239730,3.933910,2.887883,0.366073,-0.466829,2.548295,...,1.427946e-06,4.988273e-07,-5.302811e-07,-2.805575e-07,1.048873e-07,4.088274e-07,3.975385e-07,1.456760e-08,1.543444e-06,1.312034e-07
3,3,13.472752,15.214827,-7.394919,-1.805411,-3.006746,3.913534,-3.129897,-3.752737,0.368133,...,2.454887e-08,2.554858e-07,3.304949e-09,-8.249135e-08,-4.155034e-07,1.915628e-07,-1.044645e-07,1.930185e-07,1.012391e-07,-2.102301e-07
4,4,2.437661,3.493639,5.769869,3.381791,0.657089,-0.128024,5.100214,1.920374,-3.135969,...,-5.255678e-08,4.279859e-07,1.580347e-07,-1.101226e-08,-1.589480e-07,-2.772268e-07,-5.817777e-08,1.002082e-07,-2.940440e-08,3.785395e-07


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,9.468417,10.481529,-0.480864,-2.673449,1.761255,-3.715870,5.291796,4.194867,4.319843,-4.815911,...,-1.712980e-07,-7.989175e-07,-3.233028e-07,4.095239e-07,1.498896e-07,-3.064581e-07,-8.746509e-08,-2.664218e-07,2.489527e-07,-6.001070e-10
1,-0.383868,2.043244,1.357626,0.252145,4.158366,-1.648929,6.765302,2.074958,0.964495,-2.195613,...,-1.163429e-07,3.039201e-07,4.276332e-08,-5.187756e-09,2.005606e-07,7.780325e-08,-1.824773e-07,-1.162510e-07,-7.168037e-08,-2.535856e-07
2,-2.731018,2.971175,-0.750384,-0.239730,3.933910,2.887883,0.366073,-0.466829,2.548295,0.635428,...,1.427946e-06,4.988273e-07,-5.302811e-07,-2.805575e-07,1.048873e-07,4.088274e-07,3.975385e-07,1.456760e-08,1.543444e-06,1.312034e-07
3,13.472752,15.214827,-7.394919,-1.805411,-3.006746,3.913534,-3.129897,-3.752737,0.368133,1.086690,...,2.454887e-08,2.554858e-07,3.304949e-09,-8.249135e-08,-4.155034e-07,1.915628e-07,-1.044645e-07,1.930185e-07,1.012391e-07,-2.102301e-07
4,2.437661,3.493639,5.769869,3.381791,0.657089,-0.128024,5.100214,1.920374,-3.135969,-1.901910,...,-5.255678e-08,4.279859e-07,1.580347e-07,-1.101226e-08,-1.589480e-07,-2.772268e-07,-5.817777e-08,1.002082e-07,-2.940440e-08,3.785395e-07


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 200), (524, 200))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3616734445283889


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.395082,0.395082,0.617353,0.317498,0.265267,0.265267,0.259391,0.175999


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9972    1.0000    0.9986       361
           2     1.0000    0.9982    0.9991       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         0.9992      1220
   macro avg     0.9991    0.9994    0.9992      1220
weighted avg     0.9992    0.9992    0.9992      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3860    0.1401    0.2056       157
           2     0.4619    0.8692    0.6032       237
           3     0.0952    0.0154    0.0265       130

    accuracy                         0.4389       524
   macro avg     0.3144    0.3416    0.2784       524
weighted avg     0.3482    0.4389    0.3410       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.395082,0.395082,0.617353,0.317498,0.265267,0.265267,0.259391,0.175999
1,RandomForest,0.999180,0.999180,0.999183,0.999181,0.438931,0.438931,0.348175,0.341007


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5867    0.3657    0.4505       361
           2     0.5410    0.7866    0.6411       553
           3     0.5916    0.3693    0.4547       306

    accuracy                         0.5574      1220
   macro avg     0.5731    0.5072    0.5155      1220
weighted avg     0.5672    0.5574    0.5380      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2959    0.1847    0.2275       157
           2     0.4335    0.6329    0.5146       237
           3     0.1500    0.0923    0.1143       130

    accuracy                         0.3645       524
   macro avg     0.2931    0.3033    0.2854       524
weighted avg     0.3220    0.3645    0.3292       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.482787,0.482787,0.564922,0.417997,0.389313,0.389313,0.285086,0.326266
1,RandomForest,0.999180,0.999180,0.999182,0.999180,0.427481,0.427481,0.341614,0.339903
2,Adaboost,0.557377,0.557377,0.567230,0.537968,0.364504,0.364504,0.321956,0.329241


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9789    0.7701    0.8620       361
           2     0.7874    0.9982    0.8804       553
           3     0.9745    0.7484    0.8466       306

    accuracy                         0.8680      1220
   macro avg     0.9136    0.8389    0.8630      1220
weighted avg     0.8910    0.8680    0.8665      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3253    0.1720    0.2250       157
           2     0.4465    0.7046    0.5466       237
           3     0.1493    0.0769    0.1015       130

    accuracy                         0.3893       524
   macro avg     0.3070    0.3178    0.2911       524
weighted avg     0.3365    0.3893    0.3398       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.482787,0.482787,0.564922,0.417997,0.389313,0.389313,0.285086,0.326266
1,RandomForest,0.999180,0.999180,0.999182,0.999180,0.427481,0.427481,0.341614,0.339903
2,Adaboost,0.557377,0.557377,0.567230,0.537968,0.364504,0.364504,0.321956,0.329241
3,GBM,0.868033,0.868033,0.890999,0.866470,0.389313,0.389313,0.336453,0.339843


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3663    0.4211    0.3918       361
           2     0.5361    0.4702    0.5010       553
           3     0.3656    0.3824    0.3738       306

    accuracy                         0.4336      1220
   macro avg     0.4227    0.4245    0.4222      1220
weighted avg     0.4431    0.4336    0.4368      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3333    0.3758    0.3533       157
           2     0.4843    0.4557    0.4696       237
           3     0.2258    0.2154    0.2205       130

    accuracy                         0.3721       524
   macro avg     0.3478    0.3490    0.3478       524
weighted avg     0.3749    0.3721    0.3729       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.395082,0.395082,0.617353,0.317498,0.265267,0.265267,0.259391,0.175999
1,RandomForest,0.999180,0.999180,0.999183,0.999181,0.438931,0.438931,0.348175,0.341007
2,SVM,0.433607,0.433607,0.443079,0.436753,0.372137,0.372137,0.374940,0.372930


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4987    0.5402    0.5186       361
           2     0.5691    0.6926    0.6248       553
           3     0.5064    0.2582    0.3420       306

    accuracy                         0.5385      1220
   macro avg     0.5247    0.4970    0.4951      1220
weighted avg     0.5325    0.5385    0.5224      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2955    0.3312    0.3123       157
           2     0.4301    0.5190    0.4704       237
           3     0.3065    0.1462    0.1979       130

    accuracy                         0.3702       524
   macro avg     0.3440    0.3321    0.3269       524
weighted avg     0.3591    0.3702    0.3554       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.482787,0.482787,0.564922,0.417997,0.389313,0.389313,0.285086,0.326266
1,RandomForest,0.999180,0.999180,0.999182,0.999180,0.427481,0.427481,0.341614,0.339903
2,Adaboost,0.557377,0.557377,0.567230,0.537968,0.364504,0.364504,0.321956,0.329241
3,GBM,0.868033,0.868033,0.890999,0.866470,0.389313,0.389313,0.336453,0.339843
4,SVM,0.454098,0.454098,0.374613,0.288769,0.452290,0.452290,0.304749,0.285382
5,KNN,0.538525,0.538525,0.532548,0.522445,0.370229,0.370229,0.359068,0.355417


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4251    0.1967    0.2689       361
           2     0.4789    0.7794    0.5933       553
           3     0.4248    0.2124    0.2832       306

    accuracy                         0.4648      1220
   macro avg     0.4430    0.3962    0.3818      1220
weighted avg     0.4494    0.4648    0.4195      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3600    0.1720    0.2328       157
           2     0.4571    0.7426    0.5659       237
           3     0.2344    0.1154    0.1546       130

    accuracy                         0.4160       524
   macro avg     0.3505    0.3433    0.3178       524
weighted avg     0.3728    0.4160    0.3641       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.482787,0.482787,0.564922,0.417997,0.389313,0.389313,0.285086,0.326266
1,RandomForest,0.999180,0.999180,0.999182,0.999180,0.427481,0.427481,0.341614,0.339903
2,Adaboost,0.557377,0.557377,0.567230,0.537968,0.364504,0.364504,0.321956,0.329241
3,GBM,0.868033,0.868033,0.890999,0.866470,0.389313,0.389313,0.336453,0.339843
4,SVM,0.454098,0.454098,0.374613,0.288769,0.452290,0.452290,0.304749,0.285382
5,KNN,0.538525,0.538525,0.532548,0.522445,0.370229,0.370229,0.359068,0.355417
6,GNB,0.464754,0.464754,0.449430,0.419528,0.416031,0.416031,0.372770,0.364062
